In [1]:
from BorderModel import IncrementalModel, run_Incremental, sort_importances, print_importances
from BorderQuery import select_features, select_mungedata_simple, select_mungedata
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.grid_search import GridSearchCV
import datetime as dt
import matplotlib.pyplot as plt
%matplotlib inline
import pprint
import itertools
import pdb
import random
import pandas as pd
import numpy as np

### Initialize for parallel operations

In [2]:
import os
from ipyparallel import Client
rc = Client()
dview = rc[:]

# set proper working directory on all clients
cwd = os.getcwd()
dview.map(os.chdir, [cwd] * 40)
# print(dview.apply_sync(os.getcwd))

with dview.sync_imports():
    import datetime
    from BorderModel import IncrementalModel, run_Incremental
    from BorderQuery import select_features, select_mungedata_simple, select_mungedata
    from sklearn.ensemble import ExtraTreesRegressor
    from sklearn.grid_search import GridSearchCV

importing datetime on engine(s)
importing IncrementalModel,run_Incremental from BorderModel on engine(s)
importing select_features,select_mungedata_simple,select_mungedata from BorderQuery on engine(s)
importing ExtraTreesRegressor from sklearn.ensemble on engine(s)
importing GridSearchCV from sklearn.grid_search on engine(s)


In [3]:
from BorderModel import IncrementalModel, run_Incremental
with dview.sync_imports():
    from BorderModel import IncrementalModel, run_Incremental
import random

def create_train_test(year, train_length=2):
    '''
    IN 
        years: list of years to predict
        train_length: number of years to train
    '''
    train_start = datetime.date(year - train_length, 1, 1).strftime('%Y-%m-%d')
    train_end = datetime.date(year, 1, 1).strftime('%Y-%m-%d')
    test_start = datetime.date(year, 1, 1).strftime('%Y-%m-%d')
    test_end = datetime.date(year + 1, 1, 1).strftime('%Y-%m-%d')
    return train_start, train_end, test_start, test_end
    
def compare_years_parallel(model, xing, munger_id, years):
    prlist = {}
    for year in years:
        cpu = random.randint(0, 31)
        train_start, train_end, test_start, test_end = create_train_test(year, 2)

        prlist[year] = rc[cpu].apply_async(run_Incremental, model, munger_id, xing,  
                                           train_start, train_end, 
                                           test_start, test_end)
        
    return prlist

importing IncrementalModel,run_Incremental from BorderModel on engine(s)


In [4]:
def model_plot(model, start, end):
    plt.figure(figsize=(16,4))
    baseline = model.baseline()
    ensemble = model.ensemble()
    actuals = model.actual
    yhat = model.y_predict
    
    plt.plot(actuals[(actuals.index.date>=start) & (actuals.index.date<end)], label='actuals')
    plt.plot(baseline[(baseline.index.date>=start) & (baseline.index.date<end)], label='baseline')
    plt.plot(yhat[(yhat.index.date>=start) & (yhat.index.date<end)], label='predictions')
    plt.plot(ensemble[(ensemble.index.date>=start) & (ensemble.index.date<end)], label='ensemble')
    plt.legend();
    
def imp_df(xid, model_years):
    impdf = pd.DataFrame()
    for year, model in model_years.items():
        imp = sort_importances(model.model.best_estimator_, model.X.columns)
        df = pd.DataFrame(np.array(imp)[:,1], np.array(imp)[:,0]).T
        df['xid'] = xid
        df['yr'] = int(year)
        df = df.set_index(['xid', 'yr'])
        impdf = pd.concat([impdf, df])
    return impdf

## Pacific Crossing South

In [5]:
model = ExtraTreesRegressor(n_jobs=-1, n_estimators=96)
pr5 = compare_years_parallel(model, 5, 3, range(2011, 2016))

In [28]:
model5 = {}
for year in range(2011, 2016):
    if pr5[year].ready():
        model5[year] = pr5[year].get(1)
        print model5[year].score()
    else:
        print year, "not ready"

{'model': 0.50550283356833692, 'ensemble': 0.55565430134349081, 'baseline': 0.50830951929768831}
{'model': 0.6134091010507835, 'ensemble': 0.64191584560103454, 'baseline': 0.6264414468772701}
{'model': 0.61095713945601116, 'ensemble': 0.59856682003503436, 'baseline': 0.57215536973016168}
{'model': 0.57713251816633226, 'ensemble': 0.59632729083962333, 'baseline': 0.51290499913254095}
{'model': 0.321273983845897, 'ensemble': 0.32075887955528859, 'baseline': 0.23074703434185284}


In [39]:
imp5 = imp_df(5, model5)

## Pacific Crossing North

In [8]:
model = ExtraTreesRegressor(n_jobs=-1, n_estimators=96)
pr6 = compare_years_parallel(model, 6, 4, range(2013, 2016))

In [29]:
model6 = {}
for year in range(2013, 2016):
    if pr6[year].ready():
        model6[year] = pr6[year].get(1)
        print model6[year].score()
    else:
        print year, "not ready"

{'model': 0.25591339588735995, 'ensemble': 0.27747329483820438, 'baseline': 0.26915343237860456}
{'model': 0.29809133660889775, 'ensemble': 0.31232389120803694, 'baseline': 0.29165671141312366}
{'model': -0.15533941885376401, 'ensemble': -0.052260169164799919, 'baseline': -0.32952975930038231}


In [40]:
imp6 = imp_df(6, model6)

## Peace Arch South

In [11]:
model = ExtraTreesRegressor(n_jobs=-1, n_estimators=96)
pr1 = compare_years_parallel(model, 1, 3, range(2011, 2016))

In [37]:
model1 = {}
for year in range(2011, 2016):
    if pr1[year].ready():
        model1[year] = pr1[year].get(1)
        print model1[year].score()
    else:
        print year, "not ready"

{'model': 0.099495876849732601, 'ensemble': 0.38589120736921279, 'baseline': 0.39385165054016535}
{'model': 0.55228219873287654, 'ensemble': 0.61939074701522401, 'baseline': 0.60900901758379433}
{'model': 0.63965322509424571, 'ensemble': 0.63858423875556114, 'baseline': 0.61360307915900703}
{'model': 0.60024201391543197, 'ensemble': 0.6131379653881488, 'baseline': 0.56996406060235849}
{'model': 0.16913959571707027, 'ensemble': 0.1951000844239289, 'baseline': 0.1059067680212642}


In [41]:
imp1 = imp_df(1, model1)

## Peace Arch North

In [14]:
model = ExtraTreesRegressor(n_jobs=-1, n_estimators=96)
pr2 = compare_years_parallel(model, 2, 4, range(2013, 2016))

In [38]:
model2 = {}
for year in range(2013, 2016):
    if pr2[year].ready():
        model2[year] = pr2[year].get(1)
        print model2[year].score()
    else:
        print year, "not ready"

{'model': 0.38608474632848588, 'ensemble': 0.37477890721941032, 'baseline': 0.33042393202063758}
{'model': 0.41636813928593042, 'ensemble': 0.39707684796507314, 'baseline': 0.33485359571062911}
{'model': 0.10132556816543115, 'ensemble': 0.13240462589920921, 'baseline': -0.1272957779194126}


In [42]:
imp2 = imp_df(2, model2)

# Combine and compare feature importances

In [43]:
impall = pd.concat([imp1, imp2, imp5, imp6]).astype(float)

In [44]:
impall

avg_delta_1  avg_delta_10  avg_delta_11  avg_delta_12  avg_delta_2  \
xid yr                                                                         
1   2011     0.011465      0.020184      0.016488      0.015561     0.010200   
    2012     0.019971      0.016693      0.016448      0.015341     0.012885   
    2013     0.006767      0.011933      0.009765      0.008318     0.007910   
    2014     0.005978      0.008389      0.006004      0.005942     0.005951   
    2015     0.005860      0.010080      0.008382      0.008599     0.006384   
2   2013     0.006252      0.013398      0.012707      0.006940     0.009956   
    2014     0.007703      0.009313      0.007936      0.008397     0.008740   
    2015     0.007336      0.016714      0.016052      0.017187     0.009123   
5   2011     0.006696      0.020336      0.020302      0.019145     0.009511   
    2012     0.006894      0.017760      0.023338      0.015164     0.010595   
    2013     0.005176      0.009617      0.011987      0.009186     0.008932   
    2014     0.006068      0.007053      0.006597      0.007592     0.006642   
    2015     0.006151      0.008227      0.007212      0.007084     0.005899   
6   2013     0.006888      0.008065      0.008370      0.011814     0.011920   
    2014     0.008036      0.011349      0.009518      0.010209     0.008919   
    2015     0.009639      0.013893      0.018766      0.011782     0.008995   

          avg_delta_3  avg_delta_4  avg_delta_5  avg_delta_6  avg_delta_7  \
xid yr                                                                      
1   2011     0.007440     0.008448     0.007047     0.010824     0.011419   
    2012     0.010838     0.012106     0.009150     0.011173     0.009557   
    2013     0.007799     0.007316     0.008290     0.011233     0.011582   
    2014     0.008827     0.005758     0.008598     0.006821     0.009551   
    2015     0.006802     0.006986     0.009333     0.011905     0.011127   
2   2013     0.007558     0.007829     0.009700     0.015731     0.007186   
    2014     0.010419     0.008445     0.008686     0.008307     0.006699   
    2015     0.007981     0.007628     0.010111     0.007183     0.008363   
5   2011     0.006815     0.006512     0.006974     0.009625     0.011292   
    2012     0.006591     0.007106     0.006704     0.010129     0.011322   
    2013     0.006034     0.005761     0.007682     0.010279     0.010423   
    2014     0.008339     0.009026     0.008811     0.009468     0.008055   
    2015     0.007578     0.008347     0.010392     0.010408     0.010713   
6   2013     0.007965     0.006541     0.007404     0.014412     0.006016   
    2014     0.009999     0.012457     0.007893     0.010288     0.009398   
    2015     0.007738     0.010734     0.010512     0.009152     0.011474   

            ...     thunderstorm  thunderstorm_m1  thunderstorm_m2  \
xid yr      ...                                                      
1   2011    ...         0.000398         0.000475         0.002636   
    2012    ...         0.000164         0.000400         0.004226   
    2013    ...         0.000179         0.000280         0.000215   
    2014    ...         0.000253         0.000226         0.000186   
    2015    ...         0.000039         0.000019         0.000095   
2   2013    ...         0.001387         0.000470         0.000865   
    2014    ...         0.000662         0.000408         0.001243   
    2015    ...         0.000036         0.000124         0.000610   
5   2011    ...         0.000454         0.000441         0.001877   
    2012    ...         0.000161         0.000298         0.001502   
    2013    ...         0.000228         0.000148         0.000178   
    2014    ...         0.000224         0.000137         0.000160   
    2015    ...         0.000025         0.000043         0.000129   
6   2013    ...         0.001344         0.000848         0.000911   
    2014    ...         0.000265         0.000166         0.00

In [45]:
avgdelta_cols = [col for col in impall.columns.values if 'avg_delta' in col]

In [46]:
impall['trend'] = impall[avgdelta_cols].sum(1)

In [47]:
event_cols = [col for col in impall.columns.values if 'event' in col]

In [48]:
impall['event'] = impall[event_cols].sum(1)

In [49]:
impall.trend

xid  yr  
1    2011    0.143526
     2012    0.160355
     2013    0.115073
     2014    0.103250
     2015    0.111129
2    2013    0.121816
     2014    0.104195
     2015    0.133383
5    2011    0.152830
     2012    0.142802
     2013    0.111836
     2014    0.116682
     2015    0.113432
6    2013    0.107453
     2014    0.119367
     2015    0.149405
Name: trend, dtype: float64

In [50]:
impall.event

xid  yr  
1    2011    0.097278
     2012    0.119805
     2013    0.095052
     2014    0.068001
     2015    0.072147
2    2013    0.133077
     2014    0.122701
     2015    0.121989
5    2011    0.083559
     2012    0.095581
     2013    0.089317
     2014    0.066327
     2015    0.072583
6    2013    0.127205
     2014    0.115929
     2015    0.106734
Name: event, dtype: float64

In [51]:
impall.minofday

xid  yr  
1    2011    0.498811
     2012    0.483419
     2013    0.564062
     2014    0.597598
     2015    0.570534
2    2013    0.463536
     2014    0.503568
     2015    0.482101
5    2011    0.517502
     2012    0.538093
     2013    0.578162
     2014    0.578625
     2015    0.565511
6    2013    0.452839
     2014    0.494267
     2015    0.478722
Name: minofday, dtype: float64

In [52]:
impall['dayofweek']

xid  yr  
1    2011    0.057002
     2012    0.035749
     2013    0.053019
     2014    0.075991
     2015    0.077879
2    2013    0.077781
     2014    0.036059
     2015    0.040897
5    2011    0.065893
     2012    0.048127
     2013    0.062080
     2014    0.073981
     2015    0.069948
6    2013    0.079690
     2014    0.021582
     2015    0.029407
Name: dayofweek, dtype: float64

In [53]:
impall[['month', 'week', 'year']].sum(1)

xid  yr  
1    2011    0.035891
     2012    0.027484
     2013    0.026307
     2014    0.025068
     2015    0.027687
2    2013    0.041127
     2014    0.022517
     2015    0.025558
5    2011    0.036045
     2012    0.027150
     2013    0.021748
     2014    0.028939
     2015    0.042192
6    2013    0.069637
     2014    0.026480
     2015    0.027092
dtype: float64

In [54]:
weather_cols = [col for col in impall.columns.values if 'rain' in col or 'precip' in col or 'thund' in col or 
              'snow' in col or 'fog' in col or 'temp' in col or 'viz' in col or 'wind' in col]

In [55]:
impall['weather'] = impall[weather_cols].sum(1)

In [56]:
north = impall.loc[[2, 6], :]

south = impall.loc[([1, 5], [2013, 2014, 2015]),:]

In [57]:
north.trend.mean()

0.12260308269741332

In [58]:
south.trend.mean()

0.11190019745717501

In [59]:
north.event.mean()

0.12127232249014323

In [60]:
south.event.mean()

0.07723783060781812

In [61]:
north.minofday.mean()

0.4791721338135

In [62]:
south.minofday.mean()

0.5757486927023334

In [63]:
north[['dayofweek', 'month', 'week', 'year']].sum(1).mean()

0.08297104410148166

In [64]:
south[['dayofweek', 'month', 'week', 'year']].sum(1).mean()

0.09747333570488

In [65]:
south.weather.mean()

0.13763994352787054

In [66]:
north.weather.mean()

0.19398141689742232

In [67]:
north[weather_cols].mean(0)

fog                0.002835
precip             0.004553
precip_m1          0.004560
precip_m2          0.005023
precip_p1          0.006425
precip_p2          0.005320
precip_p3          0.004976
rain               0.003879
rain_m1            0.004807
rain_m2            0.003846
rain_p1            0.004046
rain_p2            0.004285
rain_p3            0.003852
snow               0.000988
snow_m1            0.001542
snow_m2            0.000529
snow_p1            0.000565
snow_p2            0.001413
snow_p3            0.000522
temp_max           0.008632
temp_max_m1        0.006348
temp_max_m2        0.006556
temp_max_p1        0.006978
temp_max_p2        0.006815
temp_max_p3        0.008452
temp_mean          0.006598
temp_mean_m1       0.005464
temp_mean_m2       0.006328
temp_mean_p1       0.006762
temp_mean_p2       0.006420
temp_mean_p3       0.006648
temp_min           0.006061
temp_min_m1        0.006502
temp_min_m2        0.007130
temp_min_p1        0.006678
temp_min_p2        0

In [68]:
north[weather_cols].mean(0)

fog                0.002835
precip             0.004553
precip_m1          0.004560
precip_m2          0.005023
precip_p1          0.006425
precip_p2          0.005320
precip_p3          0.004976
rain               0.003879
rain_m1            0.004807
rain_m2            0.003846
rain_p1            0.004046
rain_p2            0.004285
rain_p3            0.003852
snow               0.000988
snow_m1            0.001542
snow_m2            0.000529
snow_p1            0.000565
snow_p2            0.001413
snow_p3            0.000522
temp_max           0.008632
temp_max_m1        0.006348
temp_max_m2        0.006556
temp_max_p1        0.006978
temp_max_p2        0.006815
temp_max_p3        0.008452
temp_mean          0.006598
temp_mean_m1       0.005464
temp_mean_m2       0.006328
temp_mean_p1       0.006762
temp_mean_p2       0.006420
temp_mean_p3       0.006648
temp_min           0.006061
temp_min_m1        0.006502
temp_min_m2        0.007130
temp_min_p1        0.006678
temp_min_p2        0

# Top holidays

In [74]:
north[event_cols].mean().sort_values(ascending=False)

event_victoria                 0.009638
event_lead2_labor              0.004943
event_lead1_thanksgiving       0.004817
event_lead2_xmas               0.004758
event_civic                    0.004592
event_lead2_memorial           0.003637
event_lead1_labor              0.003626
event_lag1_veterans            0.003479
event_lead1_victoria           0.003214
event_lead2_newyears           0.003021
event_lag1_memorial            0.002828
event_ca_thanksgiving          0.002706
event_lead2_president          0.002516
event_lag1_independence        0.002044
event_lag4_canada              0.002013
event_lead2_independence       0.001909
event_lead1_xmas               0.001856
event_lag1_canada              0.001834
event_lag1_thanksgiving        0.001809
event_lag1_Aug_16              0.001780
event_lead3_xmas               0.001739
event_lead1_ca_thanksgiving    0.001700
event_lag1_xmas                0.001674
event_lead3_labor              0.001649
event_labor                    0.001543


In [75]:
south[event_cols].mean().sort_values(ascending=False)

event_goodfriday               0.007435
event_lag2_xmas                0.003158
event_lead2_labor              0.002782
event_lead2_civic              0.002658
event_lead1_goodfriday         0.002398
event_lead2_victoria           0.002369
event_veterans                 0.001866
event_memorial                 0.001561
event_lead2_newyears           0.001293
event_lead2_memorial           0.001243
event_lag1_newyears            0.001229
event_lead2_ca_thanksgiving    0.001132
event_lag1_xmas                0.001124
event_lead3_ca_thanksgiving    0.001027
event_lead3_independence       0.001012
event_lead4_civic              0.000987
event_canada                   0.000982
event_lag1_thanksgiving        0.000962
event_president                0.000909
event_lead1_ca_thanksgiving    0.000853
event_lag2_goodfriday          0.000815
event_thanksgiving             0.000812
event_lead1_civic              0.000732
event_xmas                     0.000716
event_lead2_xmas               0.000708


## Conclusions
August 16 is the only new holiday that has a strong effect, but not so strong as to justify addition to model.  The effect only appies in last two years, so may be ephemeral.